# Dataset editing

In [13]:
import skimage.transform
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
hira = np.load("datasets/hiragana/hiragana.npz")['arr_0'].reshape([-1, 127, 128]).astype(np.float32)

In [8]:
np.max(hira)

12.0

In [ ]:
hira = hira/np.max(hira)

In [4]:
# 71 characters, 160 writers, transform image to 48*48
train_images = np.zeros([71 * 160, 48, 48], dtype=np.float32)

for i in range(71 * 160):
    train_images[i] = skimage.transform.resize(hira[i], (48, 48))

arr = np.arange(71)
train_labels = np.repeat(arr, 160) # create labels

# split to train and test
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.2)

In [5]:
np.savez_compressed("hiragana_train_images.npz", train_images)
np.savez_compressed("hiragana_train_labels.npz", train_labels)
np.savez_compressed("hiragana_test_images.npz", test_images)
np.savez_compressed("hiragana_test_labels.npz", test_labels)

# Actual training

In [46]:
from tensorflow import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [47]:
train_images = np.load("datasets/hiragana/hiragana_train_images.npz")['arr_0']
train_labels = np.load("datasets/hiragana/hiragana_train_labels.npz")['arr_0']
test_images = np.load("datasets/hiragana/hiragana_test_images.npz")['arr_0']
test_labels = np.load("datasets/hiragana/hiragana_test_labels.npz")['arr_0']

In [48]:
if K.image_data_format() == "channels_first":
  train_images = train_images.reshape(train_images.shape[0], 1,48,48)
  test_images2 = test_images.reshape(test_images.shape[0], 1,48,48)
  shape = (1,48,48)
else:
  train_images = train_images.reshape(train_images.shape[0], 48, 48, 1)
  test_images2 = test_images.reshape(test_images.shape[0], 48, 48, 1)
  shape = (48,48,1)

In [49]:
datagen = ImageDataGenerator(rotation_range=15,zoom_range=0.2)
datagen.fit(train_images)

model = keras.Sequential([
  keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=shape),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(71, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 64)          0

In [50]:
model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

model.fit_generator(datagen.flow(train_images,train_labels,shuffle=True),epochs=30,validation_data=(test_images2,test_labels),callbacks = [keras.callbacks.EarlyStopping(patience=8,verbose=1,restore_best_weights=True),keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)])

test_loss, test_acc = model.evaluate(test_images2, test_labels)
print("Test Accuracy: ", test_acc)

Epoch 1/30


/tmp/ipykernel_29639/29902955.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(train_images,train_labels,shuffle=True),epochs=30,validation_data=(test_images2,test_labels),callbacks = [keras.callbacks.EarlyStopping(patience=8,verbose=1,restore_best_weights=True),keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)])
2024-02-19 18:10:09.732084: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-19 18:10:10.255075: I external/local_xla/xla/service/service.cc:168] XLA service 0x5d9e5a1a7f50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-19 18:10:10.255090: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-02-19 18:10:10.262098: I tensorflow/compiler/mlir/tensorflow/uti

284/284 [==============================] - 3s 4ms/step - loss: 2.3172 - accuracy: 0.3983 - val_loss: 0.6464 - val_accuracy: 0.7923 - lr: 0.0010
Epoch 2/30
284/284 [==============================] - 1s 4ms/step - loss: 0.8656 - accuracy: 0.7379 - val_loss: 0.2914 - val_accuracy: 0.9093 - lr: 0.0010
Epoch 3/30
284/284 [==============================] - 1s 4ms/step - loss: 0.5705 - accuracy: 0.8231 - val_loss: 0.2520 - val_accuracy: 0.9111 - lr: 0.0010
Epoch 4/30
284/284 [==============================] - 1s 4ms/step - loss: 0.4531 - accuracy: 0.8567 - val_loss: 0.1706 - val_accuracy: 0.9437 - lr: 0.0010
Epoch 5/30
284/284 [==============================] - 1s 4ms/step - loss: 0.3577 - accuracy: 0.8873 - val_loss: 0.1074 - val_accuracy: 0.9652 - lr: 0.0010
Epoch 6/30
284/284 [==============================] - 1s 4ms/step - loss: 0.3056 - accuracy: 0.9024 - val_loss: 0.1585 - val_accuracy: 0.9467 - lr: 0.0010
Epoch 7/30
284/284 [==============================] - 1s 4ms/step - loss: 0.2680 

In [51]:
model.save("../models/hiragana_1.keras")